In [1]:
import numpy as np
import string
import pandas as pd
from io import StringIO
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, LSTM, Dropout, Bidirectional, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


PREPARE LABELS

In [3]:
df = pd.read_json("trump_tweets_11_17.json")
replacement_dict = {"Twitter for iPhone": 1, "Twitter for Android": 0}
df = df.replace(to_replace=replacement_dict)   # replace labels
df = df[df.source.apply(lambda x: type(x) == int)]  # remove tweets from other sources

Tokenize text

In [8]:
tokenizer = Tokenizer(num_words=15000)
tokenizer.fit_on_texts(df["text"])
sequences = tokenizer.texts_to_sequences(df["text"])
text_sequences = pad_sequences(sequences, maxlen=65)
# df['tokenized_text'] = text_sequences

Split into training and testing

In [10]:
training = df.sample(frac=0.8, random_state=0)
testing = df.drop(training.index)